In [ ]:
import pymysql
import pandas as pd
# 打开数据库连接
db = pymysql.connect(host='bi-mysql-hrbidata-write.vip.sankuai.com',
                     port=5002,
                     user='Tableau',
                     password='Tableau@2021',
                     database='hrbidata')
cursor = db.cursor()
# sql2 = "SELECT * from dim_posts_info"
sql2 = """
SELECT * 
FROM dim_posts_info
WHERE createor_time >= DATE_SUB(CURDATE(), INTERVAL 7 DAY)
AND createor_time < CURDATE()
"""

cursor.execute(sql2)

results = cursor.fetchall()

# # # 打印查询结果
# for row in results:
#     print(row)
# cursor.close()

# 获取列名
columns = [desc[0] for desc in cursor.description]

# 将结果转为DataFrame
df = pd.DataFrame(results, columns=columns)

# 将DataFrame保存为CSV文件
df.to_csv('result1/2024_results.csv', index=False, encoding='utf-8')

# 创建帖子名称和内容的格式
with open('result1/posts.txt', 'w', encoding='utf-8') as f:
    for index, row in df.iterrows():
        line = f"帖子名称:{row['posts_title']}, content:{row['posts_digest']}\n"
        f.write(line)

# 关闭数据库连接
cursor.close()
db.close()

In [ ]:
# 修改read_excel函数以读取特定的sheetpip
app_id = "1681124596859981898" 
def read_excel(file_path, sheet_name):
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    return df
    # return df.iloc[:100]


def read_prompt_from_file(prompt_file_path):
    with open(prompt_file_path, 'r', encoding='utf-8') as file:
        prompt = file.read()
    return prompt


# 发送模型推理请求
def send_inference_request(prompt, app_id):
    url = "https://aigc.sankuai.com/v1/host-model/sankuai/inference"
    headers = {
        "Content-Type": "application/json;charset=UTF-8",
        "Authorization": f"Bearer {app_id}"
    }
    data = {
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "model": "LongCat-Prime-8K-Chat",  # 选择LongCat模型
        "temperature": 0,
        "max_tokens": 2048,
        "top_p": 1,
        "beam_width": 4
    }
    try:
        start_time = time.time()
        response = requests.post(url, headers=headers, data=json.dumps(data))
        end_time = time.time()
    except SSLError as e:
        logging.error(f"SSL Error occurred: {e}")
        raise Exception("SSL验证失败，请检查SSL证书。") from e

    duration = end_time - start_time
    response_json = response.json()
    # print(response.status_code)
    print(response_json)  # 打印每次返回的结果
    print(f"Response Time: {duration:.2f} seconds")  # 打印每次请求的所用时长

    if response.status_code == 200:
        response_json = response.json()
        print(response_json)
        return response_json, duration
    else:
        print(f"Request failed with status code: {response.status_code}")
        return None, duration

In [ ]:
def process_inference(file_path, app_id, result_file, prompt_file_path, output_excel_path, logger, max_retries=10):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    prompt_template = read_prompt_from_file(prompt_file_path)
    # inference_result, duration = send_inference_request(prompt_template, app_id)
    # print(inference_result)
    results = []
    output_data = []
    durations = []

    # prompt = prompt_template.replace("##话题标题和话题内容##", content)

    # print(prompt)

    # inference_result, duration = send_inference_request(prompt, app_id)

    # print(inference_result)
    # result_data = inference_result['data']['result']
    # print(result_data)

    from agent.pws import PWSBase
    from llm_config import CHAT_LLM_MODE, CHAT_LLM

    method = PWSBase(planner_model=CHAT_LLM_MODE[CHAT_LLM], solver_model=CHAT_LLM_MODE[CHAT_LLM])

    for line in content.splitlines():
        # print(line) 根据每一行内容生成一个推理请求，然后处理响应并记录结果。
        prompt = prompt_template.replace("##话题标题和话题内容##", line)
        retries = 0
        while retries < max_retries:
            try:
                # inference_result, duration = send_inference_request(prompt, app_id)
                inference_result = method.run(prompt)
                # durations.append(duration)
                if inference_result is not None:
                    try:
                        # result_data = inference_result['data']['result']
                        result_data = inference_result['output']
                        results.append(result_data)
                        output_data.append([result_data])
                        logger.info(f"Questionnaire ID:, Result: {result_data}")
                        break
                    except TypeError:
                        print("发生错误：'NoneType'对象不支持索引操作")
                        results.append("错误：无效的响应")
                        output_data.append(["错误：无效的响应"])
                        break

                else:
                    print("An error occurred: Request failed or timed out")
                    retries += 1
                    time.sleep(5)  # 等待一秒后重试

            except Exception as e:
                print(f"处理请求时发生异常: {e}")
                logger.error(f"SSL Error occurred 被抓到了: {e}")
                retries += 1
                time.sleep(5)

            if retries >= max_retries:
                print("Max retries reached. Skipping this entry.")
                results.append("Error: Unable to process")
                output_data.append(["Error: Unable to process"])
                break

    # average_duration = sum(durations) / len(durations)
    # print(f"Average Response Time: {average_duration:.2f} seconds")

    with open(result_file, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(result + '\n')

    output_df = pd.DataFrame(output_data, columns=['结果'])
    output_df.to_excel(output_excel_path, index=False)

In [ ]:
import logging
from requests.exceptions import SSLError
from logging.handlers import RotatingFileHandler
import time

In [ ]:
    # 7小时后运行的延时，单位为秒
    # delay = 7 * 60 * 60

    # 使用time.sleep()进行延时
    # time.sleep(delay)

    # keywords = ["chongtu", "gongtong", "guanxi", "xingyu", "yuanjing"]  # 存储多个keyword
    keywords = ["topic"]

    file_path = "result1/posts.txt"
    app_id = "1681124596859981898"  # 这里需要替换为实际的AppId

    for keyword in keywords:
        result_file = f"result1/topic_result.txt"
        prompt_file_path = f"prompt/fg_{keyword}_prompt_v2.txt"
        output_excel_path = f"result1/fg_{keyword}_output.xlsx"
        log_file_path = f"log/fg_{keyword}_result.log"

        # 为每个关键词创建一个新的Logger

        logger = logging.getLogger(keyword)
        logger.setLevel(logging.INFO)
        # 避免重复日志
        if not logger.handlers:
            # 创建一个FileHandler用于写入日志文件
            file_handler = RotatingFileHandler(log_file_path, maxBytes=1024 * 1024 * 100, backupCount=5,
                                               encoding='utf-8')
            file_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
            logger.addHandler(file_handler)

        # 使用logger.info等方法记录日志
        # logger.info("Your log message")

        process_inference(file_path, app_id, result_file, prompt_file_path, output_excel_path, logger=logger)

In [ ]:
import pandas as pd

# 加载CSV文件
df_csv = pd.read_csv('result1/2024_results.csv')

# 读取txt文件，假设每行是一个结果，对应CSV文件的每行
with open('result1/topic_result.txt', 'r', encoding='utf-8') as file:
    topic_results = file.readlines()

# 去除结果中的换行符
topic_results = [result.strip() for result in topic_results]

# 检查是否每行都有对应的结果
if len(topic_results) != len(df_csv):
    print("警告: txt文件中的结果数量与CSV文件中的行数不匹配。")
else:
    # 将结果作为新列添加到DataFrame中
    df_csv['result'] = topic_results

    # 保存修改后的DataFrame到新的CSV文件
    df_csv.to_csv('result1/2024_result_with_topic.csv', index=False)


In [ ]:
# 首先，过滤掉'df_csv'中'result'列值为'Error: Unable to process'的行
df_filtered = df_csv[df_csv['result'] != 'Error: Unable to process']

# 然后，将过滤后的DataFrame导出到csv文件
df_filtered.to_csv('result1/2024_result_with_topic.csv', index=False)
# df_filtered.to_excel('result/2024_result_with_topic.xlsx', index=False)


In [ ]:
import pandas as pd
import json
import re

# 加载csv文件
df = pd.read_csv('result1/2024_result_with_topic.csv')

# 定义一个辅助函数用于提取"帖子类别"和直接提取"实体"到"}"之间的所有内容
def extract_category_and_entities(x):
    category_pattern = r'"帖子类别":\s*"([^"]+)"'  # 匹配"帖子类别": "xxx"
    entities_pattern = r'"实体":\s*"([^}]+)"'  # 匹配"实体": "xxx"到"}"之间的内容

    category_match = re.search(category_pattern, x)
    entities_match = re.search(entities_pattern, x)

    category = category_match.group(1) if category_match else None
    entities = entities_match.group(1) if entities_match else None
    # 如果存在多个实体，它们将通过逗号分隔，这里将它们分割成列表
    entities_list = entities.split('","') if entities else None

    return category, entities_list

# 应用辅助函数，提取"帖子类别"和"实体"到"}"之间的所有内容
df['帖子类别'], df['实体'] = zip(*df['result'].apply(extract_category_and_entities))

# 保存到新的csv文件
df.to_csv('result1/topic_result_extracted_final.csv', index=False)


In [ ]:
import pandas as pd

# 加载csv文件
df = pd.read_csv('result1/topic_result_extracted_final.csv')

# 对“帖子类别”字段进行预处理，去掉引号等符号但保留/
df['帖子类别'] = df['帖子类别'].str.replace(r'[^\w/]', '', regex=True)

# 保存修改后的DataFrame到新的csv文件
df.to_csv('result1/topic_result_preprocessed_final.csv', index=False)


In [ ]:
import pandas as pd

# 加载xlsx文件
df = pd.read_csv('result1/topic_result_preprocessed_final.csv')


# 定义映射表
yst = {
    "客户服务": "用户体验",
    "产品建议": "建议反馈",
    "信息安全": "安全保障",
    "技术问题": "技能发展",
    "产品反馈": "建议反馈",
    "薪酬管理": "薪酬福利",
    "权限管理": "平台运营",
    "业务发展": "市场竞争",
    "业务合作": "平台合作",
    "技术分享": "技术讲座",
    "工作效率": "工作流程",
    "员工关怀": "员工福利",
    "工具使用": "办公设备",
    "业务运营": "平台运营",
    "技术讨论": "技能发展",
    "产品功能": "功能使用",
    "差旅管理": "考勤管理"
}

# 假设xlsx文件中需要映射的列名为"主题"
df['帖子类别'] = df['帖子类别'].map(yst).fillna(df['帖子类别'])

# 替换DataFrame中"帖子类别"列中所有的"为''
df['帖子类别'] = df['帖子类别'].str.replace('"', '')

# 保存修改后的DataFrame到新的xlsx文件
df.to_csv('result1/topic_result_mapped_final.csv', index=False)

In [ ]:
import pandas as pd

# 加载csv文件
df = pd.read_csv('result1/topic_result_mapped_final.csv')

# 选择posts_id和帖子类别列
category_df = df[['posts_id', '帖子类别']]

# 保存到新的csv文件
category_df.to_csv('result1/category.csv', index=False)


In [ ]:
import pandas as pd
import re

# 读取Excel文件
df = pd.read_csv('result1/topic_result_mapped_final.csv')

# 将“实体”列中所有数据先转换为字符串
df['实体'] = df['实体'].astype(str)

# 去掉“实体”列中的中括号和引号
##df['实体'] = df['实体'].apply(lambda x: re.sub(r'[\'\[\]]', '', x))
df['实体'] = df['实体'].apply(lambda x: re.sub(r'[\'\[\]":]', '', x))

# 将所有分隔符替换为英文逗号，包括中文逗号和中文顿号
df['实体'] = df['实体'].apply(lambda x: re.sub(r'[，、]', ',', x))

# 初次按逗号分割“实体”列，并展开为多行
df_expanded = df.assign(实体=df['实体'].str.split(',')).explode('实体')

# 去除拆分后的多余空格及任何符号
df_expanded['实体'] = df_expanded['实体'].apply(lambda x: x.strip())

# 对“实体”列再进行一次分割处理（如果仍然有其他分隔符存在）
df_final = df_expanded.assign(实体=df_expanded['实体'].str.split(',')).explode('实体')

# 处理后去除多余空格
df_final['实体'] = df_final['实体'].apply(lambda x: x.strip())

# 去除重复的实体
df_final = df_final.drop_duplicates(subset=['posts_id', '实体'])

# 只保留 posts_id 和 实体 两列
df_to_save = df_final[['posts_id', '实体']]

# 保存新的DataFrame到新的Excel文件
df_to_save.to_csv('result1/new_topic_result_mapped_final.csv', index=False)


In [ ]:
import pandas as pd
import re

# 假设你的Excel文件路径为'path_to_your_excel.xlsx'，entity列是你想要处理的列
# df = pd.read_excel('../data/id_entity_transformed.xlsx')
df = pd.read_csv('result1/new_topic_result_mapped_final.csv')

retain_pattern = re.compile(r'L\d+([-/]?\d+)*([-/]?\d+[A-Za-z]?)*')


# 函数：删除非汉字和字母字符，但保留规定的格式
def clean_entity(entity):
    # 确保entity是字符串
    if not isinstance(entity, str):
        entity = str(entity)
    # 去除空格
    entity = entity.replace(' ', '')
    # 检查是否匹配保留的格式
    if retain_pattern.search(entity):
        return entity
    else:
        # 删除非汉字和字母字符
        cleaned_entity = re.sub(r'[^\u4e00-\u9fa5a-zA-Z]', '', entity)
        # 剔除长度小于2的值
        if len(cleaned_entity) < 2:
            return None
        return cleaned_entity


# 对entity列进行处理
df['实体'] = df['实体'].apply(clean_entity)

# 删除值为None的行
df = df.dropna(subset=['实体'])


# 保存到新的Excel文件
df.to_csv('result1/entry.csv', index=False)

In [ ]:
import openpyxl
import re
import csv

#把实体变成规范写法
# 规范的专业术语列表
standard_terms = [
    "简称",
    "AI",
    "AIGC",
    "AM",
    "API",
    "App",
    "ARPU",
    "BA",
    "BA Copilot",
    "Bar Raiser",
    "BD",
    "BDM",
    "Better Box",
    "BG",
    "BG HR head",
    "BGFP",
    "BGM",
    "BGOD",
    "BM",
    "BP",
    "BP leader",
    "BR",
    "B-team",
    "BU",
    "bug",
    "B端",
    "C&B",
    "C2",
    "CAC",
    "Calibri",
    "CAP",
    "CC",
    "CKA",
    "CLC",
    "CMS",
    "COD",
    "CoE",
    "Copilot",
    "Correction of Error Report",
    "CPA",
    "CPC",
    "CPT",
    "CR",
    "CRM",
    "C端",
    "DAU",
    "DD",
    "Debrief",
    "EAP",
    "EFM",
    "EM",
    "Empty Chair",
    "ER",
    "FAQ",
    "final",
    "Fire",
    "Flywheel",
    "FM",
    "Forte",
    "FP",
    "GA",
    "GFM",
    "GMV",
    "GROW",
    "G-team",
    "H1",
    "H2",
    "HC",
    "HCM",
    "HCP系统",
    "HC系统",
    "Hire",
    "HRBA Copilot",
    "HRBP",
    "HRD",
    "HRIS",
    "HRM",
    "HV",
    "IC",
    "IDP",
    "Input",
    "IPH",
    "JD",
    "Job",
    "Job Family",
    "Job Group",
    "KA",
    "KPA",
    "KPI",
    "LBS",
    "Location",
    "long-term",
    "LP",
    "LRP",
    "LTI",
    "LTV",
    "MAU",
    "MBR",
    "MECE",
    "MIS",
    "MM",
    "MO",
    "MoM",
    "NSP",
    "OA",
    "OC",
    "OD",
    "Offsite Meeting",
    "OKR",
    "OLR",
    "OP",
    "OP1",
    "OP2",
    "OPR",
    "Org Review",
    "output",
    "OV",
    "PA",
    "Pay mix",
    "Payroll",
    "PDCA",
    "PECL",
    "PEST",
    "PFM",
    "Phasing",
    "PIP",
    "PM",
    "PMC",
    "POI",
    "pp",
    "PR",
    "PrOACT",
    "PV",
    "PXT",
    "QBR",
    "QoQ",
    "Range",
    "RASCI",
    "Review",
    "ROI",
    "Role Guideline",
    "SA",
    "SCQA",
    "SDE",
    "SF",
    "SKU",
    "SMART",
    "SOA",
    "SSC",
    "STAR",
    "S-Team goal",
    "STL",
    "SVP",
    "SWOT",
    "Tag",
    "Talent Discussion",
    "TB",
    "TD",
    "Team Goals",
    "Tenets",
    "The Andon Cord",
    "TPA",
    "TPM",
    "TPU",
    "TTC",
    "TTP",
    "Two-pizza Team",
    "UV",
    "VD",
    "VOC",
    "WB",
    "WBR",
    "Why",
    "WoW",
    "YoY"
]

# 创建一个空字典用于存储映射关系
term_mapping = {}
for term in standard_terms:
    term_mapping[term.lower()] = term

# 定义一个函数，用于处理包含专业术语的字符串
def standardize_term_in_context(non_standard_term):
    changed = False  # 新增变量，用于标记是否进行了转换
    for key, value in term_mapping.items():
        pattern = re.compile(re.escape(key), re.IGNORECASE)
        # 修改匿名函数，增加转换标记
        non_standard_term, n = pattern.subn(lambda m: value, non_standard_term)
        if n > 0:  # 如果发生了替换
            changed = True
    return non_standard_term, changed  # 返回转换后的字符串和是否转换的标记

with open('result1/entry.csv', mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    rows = list(reader)  # 将CSV文件的内容读取到列表中

entity_column_index = None
header = rows[0]  # CSV文件的标题行
for index, title in enumerate(header):
    if title == "实体":
        entity_column_index = index
        break

if entity_column_index is not None:
    for row_index, row in enumerate(rows):
        if row_index == 0:  # 跳过标题行
            continue
        non_standard_term = row[entity_column_index]
        if non_standard_term:
            standardized_term, changed = standardize_term_in_context(non_standard_term)
            if changed:
                print(f"行{row_index + 1}: '{non_standard_term}' 被转换为 '{standardized_term}'")
            row[entity_column_index] = standardized_term

    # 将处理后的内容写回CSV文件
    with open('result1/entry_guifan.csv', mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(rows)
else:
    print("未找到名为'实体'的列")